<a href="https://colab.research.google.com/github/RAJATkumarp/CatvsDogClassifier/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-ngrok


In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np
import cv2
import os

app = Flask(__name__)

# Load the trained models
svm_model = joblib.load('svm_model.pkl')
rf_model = joblib.load('rf_model.pkl')
lr_model = joblib.load('lr_model.pkl')  # Corrected from lt_model to lr_model
kmeans_model = joblib.load('kmeans_model.pkl')

# Load PCA and StandardScaler (if used during training)
pca = joblib.load('pca.pkl')
scaler = joblib.load('scaler.pkl')

# Function to preprocess the uploaded image
def preprocess_image(image_path, size=(64, 64)):
    # Load image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Unable to read image")
    # Resize image
    image = cv2.resize(image, size)
    # Flatten image to 1D array
    image = image.flatten()
    # Normalize pixel values
    image = image / 255.0
    return image

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'})

    file = request.files['file']
    model_name = request.form['model']

    # Save the uploaded file temporarily
    file_path = 'temp_image.jpg'
    file.save(file_path)

    try:
        # Preprocess the image
        image = preprocess_image(file_path)
        image = scaler.transform([image])  # Standardize the image
        image = pca.transform(image)  # Apply PCA (if used during training)

        # Select the model
        if model_name == 'svm':
            prediction = svm_model.predict(image)
        elif model_name == 'random_forest':
            prediction = rf_model.predict(image)
        elif model_name == 'logistic_regression':
            prediction = lr_model.predict(image)  # Corrected from lt_model to lr_model
        elif model_name == 'kmeans':
            prediction = kmeans_model.predict(image)
        else:
            return jsonify({'error': 'Invalid model selected'})  # Fixed JSON syntax

        # Map prediction to label
        label = 'cat' if prediction[0] == 0 else 'dog'
        return jsonify({'prediction': label})  # Fixed JSON syntax

    except Exception as e:
        return jsonify({'error': str(e)})  # Fixed JSON syntax

    finally:
        # Delete the temporary file
        if os.path.exists(file_path):
            os.remove(file_path)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
